# AiiDA REST API

In [ ]:
from __future__ import print_function

from IPython.display import display, clear_output
from IPython.utils import io
import ipywidgets as ipw
import multiprocessing
from urllib2 import urlopen
import json
import os
import time

In [ ]:
# starting/stopping AiiDA REST API
server_endpoint = '/api/v2'

class Toggler:
    """Provides start/stop button for a server."""
    def __init__(self, name, start_cmd, url, wait_time=0.2):
        self.name = name
        self.start_cmd = start_cmd
        self.url = url

        self.proc = None
        self.started = False
        self.start_label = "Start {}".format(name)
        self.stop_label = "Stop {}".format(name)
        self.wait_time = wait_time
        
        self.btn = ipw.Button(description=self.start_label)
        self.btn.on_click(self.toggle)
        self.info = ipw.Output()
        
    def toggle(self,b):
        self.started = not self.started
        
        if self.started:
            self.btn.description = self.stop_label
            self.proc = multiprocessing.Process(target=self.start)
            # hide output of server process
            with io.capture_output():
                self.proc.start()
            time.sleep(self.wait_time)
            with self.info:
                clear_output()
                print("Started {} at {}".format(self.name, self.public_url))
        else:
            self.btn.description = self.start_label
            self.proc.terminate()
            with self.info:
                clear_output()
                print("Stopped {}".format(self.name))

    def start(self):
        cmd = self.start_cmd
        ! $cmd
        
    @property
    def hbox(self):
        return ipw.HBox([self.btn, self.info])
    
    @property
    def public_url(self):
        return self.url
    
     
rest_toggler = Toggler(name='REST API', start_cmd='verdi restapi', url='http://127.0.0.1:5000'+server_endpoint)

display(rest_toggler.hbox)

In [ ]:
# on AiiDA Lab: provide external access via ngrok
ngrok = '/project/ngrok'
on_aiidalab = 'AIIDALAB_DOMAIN' in os.environ

class NgrokToggler(Toggler):
    @property
    def public_url(self):
        # get public urls by querying ngrok api
        response = urlopen(self.url+'/api/tunnels')
        jdict = json.load(response)
        url = jdict['tunnels'][0]['public_url']
        return url + server_endpoint
    
if on_aiidalab:
    
    ngrok_toggler = NgrokToggler(name='exposing', 
                                 start_cmd=ngrok+' http 5000 --region eu --bind-tls true', 
                                 url='http://127.0.0.1:4040',
                                 wait_time=1.0)
    display(ngrok_toggler.hbox)

In [ ]:
# testing REST endpoints

def request(b):
    with test_out:
        clear_output()
        print("GET {}".format(btn_url.value))
        response = urlopen(btn_url.value)
        print(response.read())
    
btn_url = ipw.Text(layout=ipw.Layout(width='40%'),
                   value='http://127.0.0.1:5000' + server_endpoint)
btn_test = ipw.Button(description="Test URL")
btn_test.on_click(request)

test_out = ipw.Output()
test_out.layout.border ="2px solid black"        

display(ipw.HBox([btn_test, btn_url]), test_out)